In [27]:
import torch
from torch import nn

In [28]:
inputs = torch.Tensor([[[0.2,0.1,0.3],[0.5,0.1,0.1]]])
B,S,E = inputs.size() #(batch_size,num_queries,emb_dim)
inputs = inputs.reshape(S,B,E)  #(num_queries,batch_size,emb_dim)
inputs.size()

torch.Size([2, 1, 3])

In [29]:
class LayerNormalization():
  def __init__(self,parameter_shape,eps = 1e-5):
    self.parameter_shape = parameter_shape
    self.eps = eps
    self.gamma = nn.Parameter(torch.ones(self.parameter_shape))
    self.beta = nn.Parameter(torch.zeros(self.parameter_shape))
  def forward(self,inpust):
    dims = [-(i+1) for i in range(len(self.parameter_shape))]
    mean = torch.mean(inputs,dim = dims,keepdim = True)
    var = ((inputs-mean)**2).mean(dim = dims,keepdim = True)
    std = torch.sqrt(var+self.eps)
    y = (inputs-mean)/std
    out = self.gamma * y + self.beta
    return out



In [30]:
parameter_shape = inputs.size()[-2:]

In [31]:
case1 = LayerNormalization(parameter_shape)
case1.forward(inputs)

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]], grad_fn=<AddBackward0>)